In [14]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm

In [2]:
#yahoo sports Antlers
from urllib.request import urlopen
from bs4 import BeautifulSoup
#url = 'https://soccer.yahoo.co.jp/jleague/teams/schedule/120'
html = urlopen('https://soccer.yahoo.co.jp/jleague/teams/schedule/120')
soup = BeautifulSoup(html, "html.parser")
title_tag = soup.title
title = title_tag.string
#print (title)
#print(soup)

In [3]:
#スコアのURL
import re 
score = soup.find_all(href = re.compile("^/jleague/game/score/"))
score_list = []
for v in score:
    score_list.append(v.text)

#文字化け？を修正
#一旦文字列にする。
score_list_str = ",".join(score_list).replace("\xa0","")
score_list = score_list_str.split(",")
#確認
score_list[0:10]

['1-1', '1-2', '0-0', '1-0', '0-2', '0-1', '1-1', '0-1', '0-0', '2-2']

In [4]:
#結果を格納しているタグを発見してテキストにする。
results = soup.find_all(class_="textC")
print(type(results))
#for v in results:
    #print(type(v))
    #print(v.text)

<class 'bs4.element.ResultSet'>


In [5]:
#勝敗だけ抽出。
results_list = []
for v in results:
    if (v.text == "△" or v.text == "●" or  v.text == "○" ):
        results_list.append(v.text)
    elif ("J1" in v.text):
        results_list.append(v.text)
    else:
        pass

print(results_list)

['△', '○', 'J1 第1節', '△', 'J1 第2節', '○', '○', 'J1 第3節', '●', '△', 'J1 第4節', '○', 'J1 第5節', '△', '△', 'J1 第6節', '●', 'J1 第7節', '●', 'J1 第8節', '○', '●', 'J1 第9節', '●', 'J1 第10節', '△', 'J1 第11節', '●', 'J1 第12節', '○', 'J1 第13節', '○', '○', '●', 'J1 第15節', '●', '○', '○', 'J1 第16節', '△', 'J1 第17節', '○', 'J1 第14節', '○', 'J1 第18節', '△', 'J1 第19節', '●', 'J1 第20節', '○', 'J1 第21節', '●', 'J1 第22節', '○', 'J1 第23節', '○', 'J1 第24節', '△', '○', 'J1 第25節', '●', '△', '○', 'J1 第26節', '○', '○', 'J1 第27節', '○', '○', 'J1 第28節', '○', '○', 'J1 第29節', '△', '●', '△', 'J1 第30節', '●', '△', 'J1 第31節', '○', '○', 'J1 第32節', '○', '△', '○', 'J1 第33節', '○', 'J1 第34節']


In [6]:
#リストの勝敗を数値に変換。
results_list = ["2" if "△" in s  else s for s in results_list]
results_list = ["1" if "○" in s  else s for s in results_list]
results_list = ["0" if "●" in s  else s for s in results_list]

In [7]:
#結果だけ。
pure_results = [i for i in results_list if  not "J1" in i]

In [15]:
#info of Home/Away
home_away_url = []
home_away_list = []
for a in score:
        home_away_url.append(a.get("href"))
for i in tqdm(home_away_url):
    home_away_html = urlopen("https://soccer.yahoo.co.jp/" + i )
    home_away_soup = BeautifulSoup(home_away_html, "html.parser")
    if str(home_away_soup.title.contents[0]).split()[-7] == "鹿島アントラーズ":
        home_away_list.append(1)
    else:
        home_away_list.append(0) 

In [34]:
#試合のレビューの獲得
home_away_soup.find("p",class_="", text=re.compile("。")).text

'ホーム最終戦を白星で飾りたい仙台だったが、アジア王者の鹿島に試合巧者ぶりを見せつけられる結果となった。序盤はバランスよく戦って五分五分の展開に持ち込むが、セットプレーの流れから先制点を献上し、ビハインドで折り返す。後半に入って反撃を試みるが、球際で強さを発揮する鹿島の守備陣に封じ込められてしまう。逆に、鈴木やセルジーニョを軸とする縦に鋭い攻撃から立て続けにゴールを許し、点差は3点に拡大。中野やジャーメインなど、交代選手を中心に最後まで抵抗する姿勢を見せたが及ばず、悔しい完封負けとなった。'

In [10]:
#リストの結合
df = pd.DataFrame({"results" : pure_results,
                                   "score" : score_list,
                                  "home" : home_away_list})
df.head()

,results,score,home
0,2,1-1,1
1,1,1-2,0
2,2,0-0,0
3,1,1-0,1
4,1,0-2,0


In [11]:
#csvにして出力
df.to_csv("/Users/yamadaikuya/pythonista/toto/results/antlers.csv",sep=",")